In [1]:
from IPython.core.display import HTML
from IPython.display import IFrame

#import urllib # package required to interact with live webpage
#import requests

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time
import re

In [2]:
site_url = 'https://www.grupobancolombia.com/personas/productos-servicios/inversiones/fondos-inversion-colectiva/aplicacion-fondos/'

options = Options()
options.add_argument("--headless") # To Avoid the navigator to open
#options.add_argument("--window-size=1980,1020")

browser = webdriver.Chrome(options= options)

browser.get(site_url)
time.sleep(2)
default_html = browser.page_source

#time.sleep(1)

# Ending the web driver session
#browser.quit()

In [3]:
def selecting_fund(select, value_to_select):
    '''function that selects a investment fund by value on a webpage's dropdown '''
    
    # select by value
    select.select_by_value(value_to_select)
    time.sleep(2)
    
    html = browser.page_source
    
    return html

In [4]:
soup = BeautifulSoup(default_html, 'lxml')

# Find select tag
select_tag = soup.find("select")

# find all option tag inside select tag
options = select_tag.find_all("option")
options_dict = {}

# Iterate through all option tags and get inside text
for i,option in enumerate(options):
    # Using a regular expression to extract the value of the option
    option_value = re.findall('\"([0-9]{1,2})',str(options[i]))[0]
    option_text = option.text
    options_dict[option_value] = option_text
    print(option_value,option_text)

0 Plan Semilla
1 Renta Balanceado
2 Renta Acciones
3 Renta Fija Plazo
4 Fidurenta
5 Fiducuenta
6 Fiduexcedentes
7 FICI Sura AM Rentas Inmobiliarias
8 Renta Liquidez
9 Renta Variable Colombia
10 Renta Sostenible Global
11 Renta Fija Plus
12 Renta Alta Convicción
13 Renta Futuro


In [5]:
element_dropdown = browser.find_element_by_name('nmSelectFondo')
select = Select(element_dropdown)

In [6]:
def open_excel(excel_path = 'investment_funds.xlsx'):
    saved_data = pd.read_excel(excel_path)
    return saved_data

In [114]:
historical_df = open_excel()

temp = pd.DataFrame(columns = historical_df.columns)

fund_to_select = '7'

html = selecting_fund(select, fund_to_select)

soup = BeautifulSoup(html, 'lxml')

df = pd.read_html(html)
df = pd.DataFrame(df[0])

general_info_df = df.loc[:4,:].T
general_info_df = general_info_df.loc[~general_info_df.duplicated(keep = 'first'),:].T
general_info_df.columns = ['parameter','value']
general_info_df.value.fillna('No aplica',inplace = True)

days_profitability_df = df.loc[7:8,:].T
days_profitability_df.columns = ['parameter','value']

years_profitability_df = df.loc[10:11,:].T
years_profitability_df.columns = ['parameter','value']

closing_date_df = df.loc[13:14,:].T
closing_date_df = closing_date_df.loc[~closing_date_df.duplicated(keep = 'first'),:].T
closing_date_df.columns = ['parameter','value']

fund_info = pd.concat([general_info_df,days_profitability_df,years_profitability_df,closing_date_df])

fund_info.value = fund_info.value.str.replace('$','')
fund_info.value = fund_info.value.str.replace('%','')

key_list = fund_info.parameter.tolist()
key_list.append('Fondo de Inversion')
value_list = fund_info.value.tolist()
value_list.append(options_dict[fund_to_select])
value_list = [[x] for x in value_list]
fund_info_dict = dict(zip(key_list,value_list))

#Columns to replace some character
str_columns = ['Valor de la unidad','7 días','30 días','180 días','Año corrido',
               'Último año','Últimos dos años','Últimos tres años']

current_fund_info = pd.DataFrame(data = fund_info_dict)
current_fund_info['Valor en Pesos'] = current_fund_info['Valor en Pesos'].str.replace(',','').astype('float')
current_fund_info[str_columns] = current_fund_info[str_columns].apply(lambda x: x.str.replace(',','.'), axis = 0)

current_fund_info[str_columns] = current_fund_info[str_columns].apply(lambda x : x.astype('float') 
                                                                      if ~x.str.contains('N/A').any() 
                                                                      else x.astype('object'), axis = 0)

#current_fund_info = current_fund_info.astype({'Valor de la unidad':'float64',                          
#                                              'Valor en Pesos':'float64',
#                                              '7 días':'float64',
#                                              '30 días':'float64',
#                                              '180 días':'float64',
#                                              'Año corrido':'float64',
#                                              'Último año':'float64',
#                                              'Últimos dos años':'float64',
#                                              'Últimos tres años':'float64',
#                                             })

if current_fund_info['Fondo administrador por'].str.contains('Fiduciaria').any():
    current_fund_info['Valor de la unidad'] = current_fund_info['Valor de la unidad']*1000

current_fund_info['Fecha de Cierre'] = pd.to_datetime(current_fund_info['Fecha de Cierre'], format = '%Y/%m/%d')

temp = pd.concat([temp,current_fund_info])
    
historical_df = pd.concat([historical_df, temp])
historical_df

,Fondo administrador por,Calificación,Plazo,Valor de la unidad,Valor en Pesos,7 días,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Fondo de Inversion
0,Valores Bancolombia S.A.,No aplica,No aplica,28325.72,5.254215e+11,128.61,91.36,30.28,34.98,24.35,19.55,17.37,2021-07-13,Renta Sostenible Global
0,Fiduciaria Bancolombia S.A.,No Calificado,No aplica,11755.18,4.182957e+11,7.22,12.22,4.51,4.05,1.32,8.27,N/A,2021-07-13,FICI Sura AM Rentas Inmobiliarias


In [7]:
historical_df = open_excel()

temp = pd.DataFrame(columns = historical_df.columns)

for val in options_dict.keys():
    fund_to_select = val
    
    print('Extracting information for: '+ options_dict[val])
    
    html = selecting_fund(select, fund_to_select)

    soup = BeautifulSoup(html, 'lxml')

    df = pd.read_html(html)
    df = pd.DataFrame(df[0])

    general_info_df = df.loc[:4,:].T
    general_info_df = general_info_df.loc[~general_info_df.duplicated(keep = 'first'),:].T
    general_info_df.columns = ['parameter','value']
    general_info_df.value.fillna('No aplica',inplace = True)

    days_profitability_df = df.loc[7:8,:].T
    days_profitability_df.columns = ['parameter','value']

    years_profitability_df = df.loc[10:11,:].T
    years_profitability_df.columns = ['parameter','value']

    closing_date_df = df.loc[13:14,:].T
    closing_date_df = closing_date_df.loc[~closing_date_df.duplicated(keep = 'first'),:].T
    closing_date_df.columns = ['parameter','value']

    fund_info = pd.concat([general_info_df,days_profitability_df,years_profitability_df,closing_date_df])

    fund_info.value = fund_info.value.str.replace('$','')
    fund_info.value = fund_info.value.str.replace('%','')

    key_list = fund_info.parameter.tolist()
    key_list.append('Fondo de Inversion')
    value_list = fund_info.value.tolist()
    value_list.append(options_dict[fund_to_select])
    value_list = [[x] for x in value_list]
    fund_info_dict = dict(zip(key_list,value_list))

    #Columns to replace some character
    str_columns = ['Valor de la unidad','7 días','30 días','180 días','Año corrido',
                   'Último año','Últimos dos años','Últimos tres años']

    current_fund_info = pd.DataFrame(data = fund_info_dict)
    current_fund_info['Valor en Pesos'] = current_fund_info['Valor en Pesos'].str.replace(',','').astype('float')
    current_fund_info[str_columns] = current_fund_info[str_columns].apply(lambda x: x.str.replace(',','.'), axis = 0)
    
    current_fund_info[str_columns] = current_fund_info[str_columns].apply(lambda x : x.astype('float') 
                                                                      if ~x.str.contains('N/A').any() 
                                                                      else x.astype('object'), axis = 0)
    
    #current_fund_info = current_fund_info.astype({'Valor de la unidad':'float64',                          
    #                                              'Valor en Pesos':'float64',
    #                                              '7 días':'float64',
    #                                              '30 días':'float64',
    #                                              '180 días':'float64',
    #                                              'Año corrido':'float64',
    #                                              'Último año':'float64',
    #                                              'Últimos dos años':'float64',
    #                                              'Últimos tres años':'float64',
    #                                             })

    if current_fund_info['Fondo administrador por'].str.contains('Fiduciaria').any():
        current_fund_info['Valor de la unidad'] = current_fund_info['Valor de la unidad']*1000

    current_fund_info['Fecha de Cierre'] = pd.to_datetime(current_fund_info['Fecha de Cierre'], format = '%Y/%m/%d')
    current_fund_info['Fecha Extracción']= pd.to_datetime(time.strftime('%Y/%m/%d %H:%M:%S', time.localtime(time.time())))

    temp = pd.concat([temp,current_fund_info])
    
historical_df = pd.concat([historical_df, temp]).reset_index(drop = True)

Extracting information for: Plan Semilla
Extracting information for: Renta Balanceado
Extracting information for: Renta Acciones
Extracting information for: Renta Fija Plazo
Extracting information for: Fidurenta
Extracting information for: Fiducuenta
Extracting information for: Fiduexcedentes
Extracting information for: FICI Sura AM Rentas Inmobiliarias
Extracting information for: Renta Liquidez
Extracting information for: Renta Variable Colombia
Extracting information for: Renta Sostenible Global
Extracting information for: Renta Fija Plus
Extracting information for: Renta Alta Convicción
Extracting information for: Renta Futuro


In [8]:
historical_df

,Fondo administrador por,Calificación,Plazo,Valor de la unidad,Valor en Pesos,7 días,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Fondo de Inversion,Fecha Extracción
0,Fiduciaria Bancolombia S.A.,(3/AAA),1 aÂ¦o,23092.94,9.281140e+11,-7.08,-4.41,-3.13,-2.69,1.88,4.00,4.67,2021-07-12,Plan Semilla,NaT
1,Fiduciaria Bancolombia S.A.,No Calificado,A la vista,15759.71,1.941127e+11,30.92,15.69,5.09,6.78,13.53,5.41,5.17,2021-07-12,Renta Balanceado,NaT
2,Fiduciaria Bancolombia S.A.,No Calificado,A la Vista,54357.24,5.681520e+10,149.72,29.19,-21.75,-19.48,13.14,-8.45,-5.06,2021-07-13,Renta Acciones,NaT
3,Fiduciaria Bancolombia S.A.,S5/AAAf(col),No aplica,29879.59,1.054988e+12,-14.17,-8.54,-7.71,-7.13,1.25,4.32,5.77,2021-07-12,Renta Fija Plazo,NaT
4,Fiduciaria Bancolombia S.A.,S3/AAAf(col),30 a 59 dÃ­as; 60 a 89 dÃ­as; 90 a 179dÃ­as; 1...,35129.79,1.202832e+12,-6.22,-1.91,-0.58,-0.27,1.97,3.89,4.24,2021-07-13,Fidurenta,NaT
5,Fiduciaria Bancolombia S.A.,(1/AAA),A la Vista,30876.30,1.200564e+13,3.08,0.45,0.24,0.30,1.10,2.54,2.86,2021-07-13,Fiducuenta,NaT
6,Fiduciaria Bancolombia S.A.,(1/AAA),A la vista,16406.55,1.029209e+12,3.38,0.52,0.35,0.43,1.39,2.73,3,2021-07-12,Fiduexcedentes,NaT
7,Fiduciaria Bancolombia S.A.,No Calificado,No aplica,11755.18,4.182957e+11,7.22,12.22,4.51,4.05,1.32,8.27,NaN,2021-07-13,FICI Sura AM Rentas Inmobiliarias,NaT
8,Valores Bancolombia S.A.,S1/AAAf(col),No aplica,15055.78,2.381749e+12,2.58,0.53,0.28,0.41,1.24,2.73,3.02,2021-07-13,Renta Liquidez,NaT
9,Valores Bancolombia S.A.,No aplica,No aplica,11750.97,2.991674e+11,147.08,29.55,-22.82,-19.28,12.91,-8.75,-5.27,2021-07-13,Renta Variable Colombia,NaT


In [9]:
historical_df.to_excel('investment_funds.xlsx',index = False)

In [8]:
element_dropdown = browser.find_element_by_name('nmSelectFondo')
select = Select(element_dropdown)
fund_to_select = '10'

html = selecting_fund(select, fund_to_select)

# Use html.parser to create soup
soup = BeautifulSoup(html, 'lxml')

soup.find("div",id = 'resultados')

<div id="resultados">
<!-- ngRepeat: info in ctrl.informacionCompleta --><table class="table app-fondos ng-scope" ng-repeat="info in ctrl.informacionCompleta">
<tbody><tr>
<td class="ng-binding" colspan="2">Fondo administrador por
								</td>
<td class="ng-binding" colspan="2">Valores Bancolombia S.A.
								</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Calificación</td>
<td class="ng-binding" colspan="2">N/A</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Plazo</td>
<td class="ng-binding" colspan="2">N/A</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Valor de la unidad</td>
<td class="ng-binding" colspan="2">28,325.72</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Valor en Pesos</td>
<td class="ng-binding" colspan="2">$525,421,510,656.00</td>
</tr>
<tr>
<td class="gray-bg-titles ng-binding" colspan="4">Rentabilidad</td>
</tr>
<tr>
<td class="bg-blue-featured ng-binding" colspan="4">Días</td>
</tr>
<tr>
<td class="gray-bg-titles ng-binding">7 días</td>
<td class="gr

In [9]:
browser.quit()

In [10]:
soup.find("table")

<table class="table app-fondos ng-scope" ng-repeat="info in ctrl.informacionCompleta">
<tbody><tr>
<td class="ng-binding" colspan="2">Fondo administrador por
								</td>
<td class="ng-binding" colspan="2">Valores Bancolombia S.A.
								</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Calificación</td>
<td class="ng-binding" colspan="2">N/A</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Plazo</td>
<td class="ng-binding" colspan="2">N/A</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Valor de la unidad</td>
<td class="ng-binding" colspan="2">28,325.72</td>
</tr>
<tr>
<td class="ng-binding" colspan="2">Valor en Pesos</td>
<td class="ng-binding" colspan="2">$525,421,510,656.00</td>
</tr>
<tr>
<td class="gray-bg-titles ng-binding" colspan="4">Rentabilidad</td>
</tr>
<tr>
<td class="bg-blue-featured ng-binding" colspan="4">Días</td>
</tr>
<tr>
<td class="gray-bg-titles ng-binding">7 días</td>
<td class="gray-bg-titles ng-binding">30 días</td>
<td class="gray-bg-titles ng-bindin

In [11]:
df = pd.read_html(html)
df

[                                                   0  \
 0                            Fondo administrador por   
 1                                       Calificación   
 2                                              Plazo   
 3                                 Valor de la unidad   
 4                                     Valor en Pesos   
 5                                       Rentabilidad   
 6                                               Días   
 7                                             7 días   
 8                                            128.61%   
 9                                               Años   
 10                                       Año corrido   
 11                                            34.98%   
 12  *Efectiva Anual (E.A.) Neta después de comisión.   
 13                                   Fecha de Cierre   
 
                                                    1  \
 0                            Fondo administrador por   
 1                           

In [12]:
df = pd.DataFrame(df[0])

In [13]:
df

,0,1,2,3
0,Fondo administrador por,Fondo administrador por,Valores Bancolombia S.A.,Valores Bancolombia S.A.
1,Calificación,Calificación,NaN,NaN
2,Plazo,Plazo,NaN,NaN
3,Valor de la unidad,Valor de la unidad,28325.72,28325.72
4,Valor en Pesos,Valor en Pesos,"$525,421,510,656.00","$525,421,510,656.00"
5,Rentabilidad,Rentabilidad,Rentabilidad,Rentabilidad
6,Días,Días,Días,Días
7,7 días,30 días,180 días,180 días
8,128.61%,91.36%,30.28%,30.28%
9,Años,Años,Años,Años


In [14]:
general_info_df = df.loc[:4,:].T
general_info_df = general_info_df.loc[~general_info_df.duplicated(keep = 'first'),:].T
general_info_df.columns = ['parameter','value']
general_info_df.value.fillna('No aplica',inplace = True)
general_info_df

,parameter,value
0,Fondo administrador por,Valores Bancolombia S.A.
1,Calificación,No aplica
2,Plazo,No aplica
3,Valor de la unidad,28325.72
4,Valor en Pesos,"$525,421,510,656.00"


In [15]:
days_profitability_df = df.loc[7:8,:].T
days_profitability_df.columns = ['parameter','value']
days_profitability_df

,parameter,value
0,7 días,128.61%
1,30 días,91.36%
2,180 días,30.28%
3,180 días,30.28%


In [16]:
years_profitability_df = df.loc[10:11,:].T
years_profitability_df.columns = ['parameter','value']
years_profitability_df

,parameter,value
0,Año corrido,34.98%
1,Último año,24.35%
2,Últimos dos años,19.55%
3,Últimos tres años,17.37%


In [17]:
closing_date_df = df.loc[13:14,:].T
closing_date_df = closing_date_df.loc[~closing_date_df.duplicated(keep = 'first'),:].T
closing_date_df.columns = ['parameter','value']
closing_date_df

,parameter,value
13,Fecha de Cierre,2021/07/13


In [18]:
fund_info = pd.concat([general_info_df,days_profitability_df,years_profitability_df,closing_date_df])
fund_info

,parameter,value
0,Fondo administrador por,Valores Bancolombia S.A.
1,Calificación,No aplica
2,Plazo,No aplica
3,Valor de la unidad,28325.72
4,Valor en Pesos,"$525,421,510,656.00"
0,7 días,128.61%
1,30 días,91.36%
2,180 días,30.28%
3,180 días,30.28%
0,Año corrido,34.98%


In [19]:
fund_info.value = fund_info.value.str.replace('$','')
fund_info.value = fund_info.value.str.replace('%','')

In [20]:
key_list = fund_info.parameter.tolist()
key_list.append('Fondo de Inversion')
value_list = fund_info.value.tolist()
value_list.append(options_dict[fund_to_select])
value_list = [[x] for x in value_list]
fund_info_dict = dict(zip(key_list,value_list))
fund_info_dict

{'Fondo administrador por': ['Valores Bancolombia S.A.'],
 'Calificación': ['No aplica'],
 'Plazo': ['No aplica'],
 'Valor de la unidad': ['28325.72'],
 'Valor en Pesos': ['525,421,510,656.00'],
 '7 días': ['128.61'],
 '30 días': ['91.36'],
 '180 días': ['30.28'],
 'Año corrido': ['34.98'],
 'Último año': ['24.35'],
 'Últimos dos años': ['19.55'],
 'Últimos tres años': ['17.37'],
 'Fecha de Cierre': ['2021/07/13'],
 'Fondo de Inversion': ['Renta Sostenible Global']}

In [21]:
current_fund_info = pd.DataFrame(data = fund_info_dict)
current_fund_info['Valor en Pesos'] = current_fund_info['Valor en Pesos'].str.replace(',','')
current_fund_info

,Fondo administrador por,Calificación,Plazo,Valor de la unidad,Valor en Pesos,7 días,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Fondo de Inversion
0,Valores Bancolombia S.A.,No aplica,No aplica,28325.72,525421510656.00,128.61,91.36,30.28,34.98,24.35,19.55,17.37,2021/07/13,Renta Sostenible Global


In [22]:
current_fund_info = current_fund_info.astype({'Valor de la unidad':'float64',                          
                          'Valor en Pesos':'float64',
                          '7 días':'float64',
                          '30 días':'float64',
                          '180 días':'float64',
                          'Año corrido':'float64',
                          'Último año':'float64',
                          'Últimos dos años':'float64',
                          'Últimos tres años':'float64',
                         })
current_fund_info['Fecha de Cierre'] = pd.to_datetime(current_fund_info['Fecha de Cierre'], format = '%Y/%m/%d')
current_fund_info

,Fondo administrador por,Calificación,Plazo,Valor de la unidad,Valor en Pesos,7 días,30 días,180 días,Año corrido,Último año,Últimos dos años,Últimos tres años,Fecha de Cierre,Fondo de Inversion
0,Valores Bancolombia S.A.,No aplica,No aplica,28325.72,5.254215e+11,128.61,91.36,30.28,34.98,24.35,19.55,17.37,2021-07-13,Renta Sostenible Global


In [23]:
current_fund_info.dtypes

Fondo administrador por            object
Calificación                       object
Plazo                              object
Valor de la unidad                float64
Valor en Pesos                    float64
7 días                            float64
30 días                           float64
180 días                          float64
Año corrido                       float64
Último año                        float64
Últimos dos años                  float64
Últimos tres años                 float64
Fecha de Cierre            datetime64[ns]
Fondo de Inversion                 object
dtype: object

In [24]:
current_fund_info.to_excel('investment_funds.xlsx',index = False)